In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('..')

In [3]:
import random

import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

/home/alm/miniforge3/envs/assistant_nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from stat_lm import Tokenizer, StatLM, GenerationConfig

In [5]:
data = load_dataset('Den4ikAI/russian_dialogues', split="train[:30%]")

In [6]:
train_texts = []
test_texts = []

data_len = len(data)

train_part = int(0.95*data_len)

for i in tqdm(range(train_part)):
    d = data[i]
    if d['relevance'] == 1:
        text = f"{d['question']} {d['answer']}"
        train_texts.append(text)

test_texts = []
for i in tqdm(range(train_part, data_len)):
    d = data[i]
    if d['relevance'] == 1:
        text = f"{d['question']} {d['answer']}"
        test_texts.append(text)

  0%|          | 0/706036 [00:00<?, ?it/s]

100%|██████████| 37160/37160 [00:01<00:00, 27510.29it/s]


In [7]:
for i in range(10):
    index = int(len(train_texts)*random.random())
    print(train_texts[index])

можно ли смотреть телевизор через компьютер? да tv тюнер надо покупать
я считаю, что лучше быть грешным, чем слыть грешным. кто согласен и почему? грехи разные, как и слава. поступайте разумно.
когда делаешь упражнения на трицепс, руку не полностью выпрямлять под конец? да, чтобы нагрузка с трицепса не уходила.
это правда что рубин обыграл барселону, как это произошло? старание и труд помноженное на везение творят чудеса. советую посмотреть матч
. можно так сильно у пасть что бы по купить любовь? про дать? а толку. любой товар имеет срок годности.
если любимый человек, после обиды становится противен, значит любви никакой и не было? а любовь это не всегда сладкая вата. иногда и горький мед.
а я люблю рубу и собак и есть за что. верные. хвостом виляют
докажи кинь член тебе своего мало?
а не кому, сейчас вообще не знаешь кому верить, истории даже переписывают вся наша история наверное ложь
ваша вера вас спасла? вера во что? вера в свои силы. в свой дух, в свою разумность.


In [8]:
all_texts = train_texts + test_texts

In [9]:
tokenizer = Tokenizer().build_vocab(all_texts)

In [10]:
Tokenizer()

In [11]:
len(tokenizer.vocab)

303535

In [12]:
text_example = "В России люди любят искать приключения на голову"

In [13]:
tokenizer.encode(text_example)

[230340, 1714, 23623, 90703, 56427, 52295, 130223, 86695, 303532]

In [14]:
tokenizer._tokenize(text_example, append_eos_token=False)

['в', 'россии', 'люди', 'любят', 'искать', 'приключения', 'на', 'голову']

In [15]:
tokenizer.decode(tokenizer.encode(text_example), remove_special_tokens=True)

'в россии люди любят искать приключения на голову'

In [16]:
stat_lm = StatLM(tokenizer, context_size=3, alpha=0.0001)

In [17]:
# "обучаем" модель - считаем статистики
stat_lm.train(train_texts)

In [18]:
config = {
    'temperature': 1.2,
    'max_tokens': 30,
    'sample_top_p': 5,
    'decoding_strategy': 'top-p',
    'gen_decay': 1,
}

generation_config = GenerationConfig(temperature=config['temperature'],
                                     max_tokens=config['max_tokens'],
                                     sample_top_p=config['sample_top_p'],
                                     decoding_strategy=config['decoding_strategy'],
                                     gen_decay=config['gen_decay'],
                                     remove_special_tokens=True)

In [19]:
for i in range(20):
    index = train_part + int(len(test_texts)*random.random())
    d = data[index]
    if d['relevance'] == 1:
        text = d['question']
        generated = stat_lm.generate_text(text, generation_config)
        print(generated['total_text'])
        print(generated['finish_reason'], '\n')

а я просто степан, музыку пишу дратуйти qcy qcy торопитесь
end of text 

а правда ли что сколько людей столько и мнений? почему? да, но не всегда. но не все, что это может быть? если нет,
max tokens 

осознание того, что ты желанен придает тебе уверенности? знания, и не надо. а если в семье. а вот интересно, а не с чем связано
max tokens 

кстати, если вы знаете хороших людей, то просьба использовать сарафанное радио и рассказать о чате у славы о, да и то же? если нет, не
max tokens 

а вот как актер вицин чем больше тем лучше. а что для вас значит? что бы вы хотели бы вы хотели бы вы хотели бы вы хотели бы
max tokens 

мелировка волос это красиво, или уже не модно? это когда не надо, чтобы не было. а вот в этом году? если да, то как
max tokens 

серьезно ждешь? я то серьезно могу кинуть если надо. а у вас? и почему? я не знаю, но не знаю, я не хочу быть
max tokens 

верите ли вы сонникам? верите что можно приготовить из куриного филе, можно ли считать это безусловным добром
end of tex

In [20]:
generated = stat_lm.generate_text("Как дела?", generation_config)
print(generated['total_text'])
print(generated['finish_reason'], '\n')

как дела? хорошо отношусь. это не значит, что бы не стал. эт про чтото интересное было, а вот если бы я был не прав.
max tokens 



In [21]:
tokenizer.save('tokenizer.pkl')

True

In [22]:
stat_lm.save_stat('stat_lm.pkl')

True